In [36]:
#install torch
%pip install torch==2.2.0 torchvision torchaudio


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/pip/__main__.py", line 24, in <module>
    sys.exit(_main())
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 114, in _main
    options, args = self.parse_args(args)
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 96, in parse_args
    return self.parser.parse_args(args)
  File "/opt/conda/lib/python3.10/optparse.py", line 1371, in parse_args
    values 

In [82]:
import json
import torch
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import f1_score
from tqdm import tqdm, trange

In [83]:
train = json.load(open('/kaggle/input/data-json/train_file.json'))
test = json.load(open('/kaggle/input/data-json/val_file.json'))

print('train:', len(train))
print('test:', len(test))

train: 6740
test: 843


## BERT ENCODINGS

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  


## DATALOADERS

# Convert json to csv

In [85]:
dialogue_ids = []
speaker = []
emotion = []
utterance = []
erf_label = []

for i in range(len(train)):
    dialogue_ids.append(train[i]['episode'])
    speaker.append(train[i]['speakers'])
    emotion.append(train[i]['emotions'])
    utterance.append(train[i]['utterances'])
    erf_label.append(train[i]['triggers'])
    # for j in range(len(train[i]['speakers'])):
    #     dialogue_ids.append(train[i]['episode'])
    #     speaker.append(train[i]['speakers'][j])
    #     emotion.append(train[i]['emotions'][j])
    #     utterance.append(train[i]['utterances'][j])
    #     erf_label.append(train[i]['triggers'][j])
df_train = pd.DataFrame(list(zip(dialogue_ids, speaker, emotion, utterance, erf_label)), columns =['Dialogue_ID', 'Speaker', 'Emotion', 'Utterance', 'ERF_Label'])
# save this dataframe to a csv file
df_train.to_csv('train.csv', index=False)
dialogue_ids = []
speaker = []
emotion = []
utterance = []
erf_label = []

for i in range(len(test)):
    dialogue_ids.append(train[i]['episode'])
    speaker.append(train[i]['speakers'])
    emotion.append(train[i]['emotions'])
    utterance.append(train[i]['utterances'])
    erf_label.append(train[i]['triggers'])
    # for j in range(len(test[i]['speakers'])):
    #     dialogue_ids.append(test[i]['episode'])
    #     speaker.append(test[i]['speakers'][j])
    #     emotion.append(test[i]['emotions'][j])
    #     utterance.append(test[i]['utterances'][j])
    #     erf_label.append(test[i]['triggers'][j])
df_test = pd.DataFrame(list(zip(dialogue_ids, speaker, emotion, utterance, erf_label)), columns =['Dialogue_ID', 'Speaker', 'Emotion', 'Utterance', 'ERF_Label'])
# save this dataframe to a csv file
df_test.to_csv('test.csv', index=False)


In [86]:
df_train.head()

,Dialogue_ID,Speaker,Emotion,Utterance,ERF_Label
0,utterance_3492,"[Phoebe, Eric, Phoebe, Eric, Phoebe]","[surprise, fear, surprise, sadness, disgust]","[You-you you had sex with Ursula?!, Uh, a litt...","[1.0, 1.0, 0.0, 0.0, 0.0]"
1,utterance_3952,"[Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...","[disgust, disgust, anger, sadness, surprise, a...","[Dad, please don't pick your teeth out here!, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,utterance_3198,"[Older Scientist, Ross, Ross, Joey, Ross, Ross...","[neutral, neutral, neutral, neutral, neutral, ...","[Dr. Geller, there's a seat over here., Thank ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]"
3,utterance_2834,"[Monica, Monica, Monica]","[neutral, surprise, neutral]","[So, how'd the lasagne go over?, Really?!, Good.]","[0.0, 0.0, 1.0]"
4,utterance_453,"[Kate, The Director, Kate]","[joy, sadness, sadness]","[Become a drama critic!, I am hurt! A plague ...","[0.0, 0.0, 1.0]"


In [87]:
df_test.head()

,Dialogue_ID,Speaker,Emotion,Utterance,ERF_Label
0,utterance_3492,"[Phoebe, Eric, Phoebe, Eric, Phoebe]","[surprise, fear, surprise, sadness, disgust]","[You-you you had sex with Ursula?!, Uh, a litt...","[1.0, 1.0, 0.0, 0.0, 0.0]"
1,utterance_3952,"[Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...","[disgust, disgust, anger, sadness, surprise, a...","[Dad, please don't pick your teeth out here!, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,utterance_3198,"[Older Scientist, Ross, Ross, Joey, Ross, Ross...","[neutral, neutral, neutral, neutral, neutral, ...","[Dr. Geller, there's a seat over here., Thank ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]"
3,utterance_2834,"[Monica, Monica, Monica]","[neutral, surprise, neutral]","[So, how'd the lasagne go over?, Really?!, Good.]","[0.0, 0.0, 1.0]"
4,utterance_453,"[Kate, The Director, Kate]","[joy, sadness, sadness]","[Become a drama critic!, I am hurt! A plague ...","[0.0, 0.0, 1.0]"


In [88]:
unicode_mapping = {}
unicode_mapping['\x85'] = '...' 
unicode_mapping['\x91'] = "'"
unicode_mapping['\x92'] = "'"
unicode_mapping['\x93'] = '"'
unicode_mapping['\x94'] = '"'
unicode_mapping['\x97'] = '--'

unicode_mapping['\u2014'] = '--'
unicode_mapping['\u2019'] = "'"
unicode_mapping['\u2026'] = '...'

unicode_mapping['\xe9'] = 'e'

def clean_utterance(utterance_list):
    '''	
    This function takes a list of utterances and replaces the unicode with the proper characters.
    '''
    cleaned_utterances_list = []
    for utterance in utterance_list:
        for key in unicode_mapping:
            utterance = utterance.replace(key, unicode_mapping[key])
        cleaned_utterances_list.append(utterance)
    return cleaned_utterances_list

train_uttr = df_train['Utterance'].apply(lambda x: clean_utterance(x))
df_train['Utterance'] = train_uttr
test_uttr = df_test['Utterance'].apply(lambda x: clean_utterance(x))
df_test['Utterance'] = test_uttr
for i in range(len(df_train)):
    for j in range(len(df_train['ERF_Label'][i])):
        if df_train['ERF_Label'][i][j] != 1.0 and df_train['ERF_Label'][i][j] != 0.0:
            df_train['ERF_Label'][i][j] = -1.0
    
    # for j in range(len(df_test['ERF_Label'][i])):
    #     if df_test['ERF_Label'][i][j] == 'None':
    #         df_test['ERF_Label'][i][j] = -1

for i in range(len(df_test)):
    for j in range(len(df_test['ERF_Label'][i])):
        if df_test['ERF_Label'][i][j] != 1.0 and df_test['ERF_Label'][i][j] != 0.0:
            df_test['ERF_Label'][i][j] = -1.0

In [89]:
print(df_train['Utterance'][0])

['You-you...you had sex with Ursula?!', 'Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and', "You didn't notice she was wearing different clothes?!", 'Well I was just so excited to see you.', "Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird."]


In [90]:
# max length episode
max_len = 0
for i in range(len(df_train)):
    max_len = max(max_len, len(df_train['Utterance'][i]))
print(max_len)

24


In [91]:
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn


tokenizer = AutoTokenizer.from_pretrained("roberta-base")


In [92]:
class Model(nn.Module):
    #model consists of a EmoBERTa model and a linear layer for sequence labeling task
    def __init__(self, num_classes=2):
        super(Model, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask)
        out = outputs[0]
        out = out[:, 0, :]
        out = self.fc(out)
        # make sure output is either 0 or 1
        out = torch.sigmoid(out)
        out = torch.round(out)
        return out
class Dataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = self.process(data, tokenizer)
        self.max_len = max_len

    def __len__(self):
        return len(self.data['input'])

    def __getitem__(self, index):
        #tokenize the input
        input = self.data['input'][index]
        target = self.data['target'][index]
#         if target == 'None':
#             target = -1
#         print(target)
        encoding = self.tokenizer(input, return_tensors='pt', padding='max_length', max_length=self.max_len, truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'target':torch.tensor(target)}
        
    def process(self, data, tokenizer):
        new_data = {'input': [], 'target': []}
        for i in range(len(data)):
            newinput = data.loc[i].copy()
            stringlist = []
            for j in range(len(newinput['Utterance'])):
                stringlist.append(newinput['Speaker'][j] + ': ' + newinput['Utterance'][j] + ":" + newinput['Emotion'][j])
            for j in range(len(newinput['Utterance'])):
                #join till jth utterance
                temp = ' '.join(stringlist[:j])
                temp += '</s></s>'
                #add jth utterance
                temp += newinput['Speaker'][j] + ': ' + newinput['Utterance'][j] + ":" + newinput['Emotion'][j]
                temp += '</s></s>'
                #add later Utterence
                temp += ' '.join(stringlist[j+1:])
                temp = '<s> ' + temp + '</s>'
                new_data['input'].append(temp)
                target = newinput['ERF_Label'][j]
                # print(target)
                new_data['target'].append(target)
        return new_data
            
    

In [93]:
def collate_fn(batch):
    input_ids = []
    attention_mask = []
    target = []
    for b in batch:
        input_ids.append(b['input_ids'])
        attention_mask.append(b['attention_mask'])
        target.append(b['target'])
    input_ids = torch.stack(input_ids)
    attention_mask = torch.stack(attention_mask)
    target = torch.stack(target)
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}

In [94]:
# take 10 samples only
# df_train = df_train[:100]
# df_test = df_test[:100]

train_Dataset = Dataset(df_train, tokenizer, 2)
train_loader = DataLoader(train_Dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

test_Dataset = Dataset(df_test, tokenizer, 2)
test_loader = DataLoader(test_Dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

model = Model(1)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
print(len(train_loader), len(test_loader))


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1843 229


In [95]:
print(device)
def train(model,train_loader,test_loader,criterion,optimizer,num_class,epochs):
    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader):
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target = batch['target'].to(device)
            # print(input_ids)
#             print(input_ids.shape, attention_mask.shape, target.shape)
            optimizer.zero_grad()
            output = model(input_ids, attention_mask)
#             print(output[0], target)
            #make output
            # output = output.unsqueeze(0) 
#             print(output.shape, target.shape)
#             loss = criterion(output, target)
            loss = criterion(output.view(-1), target)
#             print(loss.item())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        # break
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss}')
        model.eval() 
        test_loss = 0
        with torch.no_grad():
            for batch in tqdm(test_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                target = batch['target'].to(device)
                # print(input_ids)
                # print(input_ids.shape, attention_mask.shape, target.shape)
                optimizer.zero_grad()
                output = model(input_ids, attention_mask)
                # print(output, target)
                #make output
                # output = output.unsqueeze(0) 
                # print(output.shape, target.shape)
                # break 
#                 loss = criterion(output, target)
                loss = criterion(output.view(-1), target)
                f1_micro = f1_score(target.cpu().numpy(), output.cpu().numpy(), average='micro')
                f1_macro = f1_score(target.cpu().numpy(), output.cpu().numpy(), average='macro')
                f1_weighted = f1_score(target.cpu().numpy(), output.cpu().numpy(), average='weighted')
                test_loss += loss.item()
            test_loss /= len(test_loader)
            test_losses.append(test_loss)
            print(f'Epoch {epoch+1}/{epochs}, Test Loss: {test_loss},F1 micro:{f1_micro}, F1 macro:{f1_macro},F1 weighted:{f1_weighted}')

cuda


In [ ]:
train(model,train_loader,test_loader,criterion,optimizer,18,5)

100%|██████████| 1843/1843 [02:12<00:00, 13.95it/s]


Epoch 1/5, Train Loss: 17.543844316164826


100%|██████████| 229/229 [00:09<00:00, 25.31it/s]


Epoch 1/5, Test Loss: 17.945683185710656,F1 micro:0.7857142857142857, F1 macro:0.44,F1 weighted:0.6914285714285714


100%|██████████| 1843/1843 [02:11<00:00, 14.07it/s]


...

100%|██████████| 1843/1843 [02:10<00:00, 14.09it/s]


Epoch 3/5, Train Loss: 17.548074731086817


100%|██████████| 229/229 [00:08<00:00, 25.65it/s]


Epoch 3/5, Test Loss: 17.945683275247767,F1 micro:0.7857142857142857, F1 macro:0.44,F1 weighted:0.6914285714285714


 49%|████▉     | 902/1843 [01:04<01:07, 14.01it/s]